In [ ]:
import scipy as scipy
import matplotlib.pyplot as plt
import sympy as sp
import numpy as np
from ngsolve import *
from netgen.occ import *
from netgen.occ import X, Y, Z
from netgen.occ import OCCGeometry
from ngsolve.webgui import Draw
from datetime import datetime

In [ ]:
# setup mesh & geometry
# gebiet: periodisch (1.5x1.5) Quadrat, Ursprung in der Mitte => dh. mesh periodish

# ---------------------------------------
# 1. Periodische Geometrie, Mesh (Listing 5)
# ---------------------------------------


l_dom = 1.5 # Laenge des Quadrats
n_dom = 75 # Anzahl Elemente pro Kante
rec = MoveTo(-l_dom/2,-l_dom/2).Rectangle(l_dom,l_dom).Face()

# Namen der Kanten
rec.edges.Max(X).name = 'right'
rec.edges.Min(X).name = 'left'
rec.edges.Max(Y).name = 'top'
rec.edges.Min(Y).name = 'bottom'

# Identifizierung links <-> rechts
right=rec.edges.Max(X)
rec.edges.Min(X).Identify(right,name="left")

# Identifizierung oben <-> unten
top=rec.edges.Max(Y)
rec.edges.Min(Y).Identify(top,name="bottom")

# Geometrie erstellen
geo= OCCGeometry(rec,dim=2)

# Mesh generieren
mesh = Mesh(geo.GenerateMesh(maxh=l_dom/n_dom))

# print mesh info
print("Mesh info:")
print(f" Number of elements: {mesh.ne}")
print(f" Number of vertices: {mesh.nv}")
Draw(mesh)

In [ ]:
# ---------------------------------------
# 2. Finite-Elemente-Raum (Listing 2)
# ---------------------------------------
order = 3
V = Periodic(H1(mesh, order=order))
X = V * V
u, v = X.TrialFunction()
w, q = X.TestFunction()

print("Finite-Element space defined.")
print("Number of DOFs:", X.ndof)


In [ ]:
# ---------------------------------------
# 3. GridFunctions und Startwerte (Listing 3 + 1)
# ---------------------------------------
gfx = GridFunction(X)
gfu, gfv = gfx.components

gfxold = GridFunction(X)
gfuold, gfvold = gfxold.components

# Startwerte: zwei Regionen
l_init = l_dom / n_dom * 20

gfuold.Set(IfPos((l_init/2)**2 - x*x,
                 IfPos((l_init/2)**2 - y*y, 0.5, 1), 1))
gfvold.Set(IfPos((l_init/2)**2 - x*x,
                 IfPos((l_init/2)**2 - y*y, 0.25, 0), 0))

# kleine Zufallsstörung
gfxold.vec[:] += 0.01 * np.random.normal(size=X.ndof)
gfx.vec.data = gfxold.vec

Draw(gfuold, mesh, "u0")
Draw(gfvold, mesh, "v0")

print("Initial values set and visualized.")


In [ ]:
# ---------------------------------------
# 4. Parameter & Bilinearformen (Listings 4 & 6 & 7)
# ---------------------------------------
eps1 = Parameter(2e-5)
eps2 = Parameter(1e-5)

# Change this for different patterns

F = Parameter(0.056)
k = Parameter(0.065)

a = BilinearForm(X, symmetric=True)
a += eps1 * grad(u) * grad(w) * dx
a += eps2 * grad(v) * grad(q) * dx
a.Assemble()

m = BilinearForm(X, symmetric=True)
m += u*w*dx
m += v*q*dx
m.Assemble()

# print("Matrix A:\n", a.mat)
# print("Matrix M:\n", m.mat)

# Zeitparameter
dt = 10

In [ ]:
def assemble_f(gfu_src, gfv_src, target_vec):
    f_temp = LinearForm(X)
    f_temp += dt*(-gfu_src*gfv_src**2 + F*(1-gfu_src))*w*dx(bonus_intorder=4)
    f_temp += dt*(gfu_src*gfv_src**2 - (k+F)*gfv_src)*q*dx(bonus_intorder=4)
    f_temp.Assemble()
    target_vec.data = f_temp.vec
    return f

In [ ]:
# LinearForm for nonlinear terms
f = LinearForm(X)
f += dt*(-gfuold*gfvold**2+F*(1-gfuold))*w*dx(bonus_intorder=4)
f += dt*(gfuold*gfvold**2-(k+F)*gfvold)*q*dx(bonus_intorder=4)

# Visualization
scene_u = Draw(gfu, mesh, "u")
scene_v = Draw(gfv, mesh, "v")

# Hilfsvektoren
deltauv = gfx.vec.CreateVector()

# Initialize Trapez method (passing gfxold which contains gfuold, gfvold)

sgfu, gfv = gfx.components
gfuold, gfvold = gfxold.components

# M* = M + dt*A
mstar = a.mat.CreateMatrix()
mstar.AsVector().data = m.mat.AsVector() + (dt/2.0) * a.mat.AsVector()
mstarinv = mstar.Inverse(inverse="sparsecholesky")
print("System matrices assembled (M*, M, A).")


res = gfx.vec.CreateVector()
f_old = gfx.vec.CreateVector()
f_new = gfx.vec.CreateVector()
rhs = gfx.vec.CreateVector()

 # LinearForms for nonlinear terms
f_form_old = LinearForm(X)
f_form_new = LinearForm(X)


# Run simulation
nsteps = 900
max_newton_iter = 10



with TaskManager():
    for j in range(nsteps):

        #newtoniters, norm = step
        assemble_f(gfuold, gfvold, f_old)
        # Initial guess: use old solution
        gfx.vec.data = gfxold.vec
        for newton_iter in range(max_newton_iter):
            assemble_f(gfu, gfv, f_new)
            # Right hand side for trapezoidal rule:
            # RHS = M*y_n - dt/2*A*y_n + dt/2*(f_new + f_old)
            rhs.data = m.mat * gfxold.vec
            
            # Subtract dt/2 * A * y_n
            res.data = a.mat * gfxold.vec
            rhs.data -= (dt/2.0) * res
            
            # Add dt/2 * (f_new + f_old)
            res.data = f_new + f_old
            rhs.data += 0.5 * res
            
            # Solve: (M + dt/2*A) * y_{n+1} = RHS
            gfx.vec.data = mstarinv * rhs
            
            # Compute change for convergence check
            deltauv.data = gfx.vec - gfxold.vec
            norm = deltauv.Norm()
            
            # Check convergence
            if newton_iter > 0 and norm < 1e-8:
                # Update for next time step
                gfxold.vec.data = gfx.vec
                break
        gfxold.vec.data = gfx.vec

        if j % 10 == 0:
            scene_u.Redraw()
            scene_v.Redraw()
        deltauvNorm = deltauv.Norm()
        print(j,deltauvNorm,end='\r')
        # Abbruch, wenn sich nichts mehr aendert
        if deltauvNorm < 1e-8:
            print('stopping, no change in solution')
            break



In [ ]:
# ---------------------------------------
# 5. Zeitintegration (Listing 8 & 9)
# ---------------------------------------
# M* = M + dt*A
mstar = a.mat.CreateMatrix()
mstar.AsVector().data = m.mat.AsVector() + dt * a.mat.AsVector()
mstarinv = mstar.Inverse(inverse="sparsecholesky")
print("System matrices assembled (M*, M, A).")



f = LinearForm(X)
f += dt*(-gfuold*gfvold**2+F*(1-gfuold))*w*dx(bonus_intorder=4)
f += dt*(gfuold*gfvold**2-(k+F)*gfvold)*q*dx(bonus_intorder=4)

scene_u = Draw(gfu)
scene_v = Draw(gfv)

res = gfx.vec.CreateVector()
deltauv = gfx.vec.CreateVector()

nsteps = 900
with TaskManager():
    for j in range(nsteps):
        # Assemble right-hand side f (explicit nonlinearity)
        f.Assemble()

        # Solve M* δ = -dt*A*u_old + f
        res.data = -dt * a.mat * gfxold.vec + f.vec
        deltauv.data = mstarinv * res
        gfx.vec.data = gfxold.vec + deltauv

        # Update for next step
        gfxold.vec.data = gfx.vec

        # Visualize every 10 steps
        if j % 10 == 0:
            scene_u.Redraw()
            scene_v.Redraw()
        deltauvNorm = deltauv.Norm()
        print(j,deltauvNorm,end='\r')
        # Abbruch, wenn sich nichts mehr aendert
        if deltauvNorm < 1e-8:
            print('stopping, no change in solution')
            break


print("\nSimulation finished.")


In [ ]:
# ---------------------------------------
# 6. Ergebnisplot (Listing 10)
# ---------------------------------------
Npixel = 800
xi = np.linspace(-l_dom/2, l_dom/2, Npixel)
Xi, Yi = np.meshgrid(xi, xi)
mips = mesh(Xi.flatten(), Yi.flatten())

ui = gfu(mips).reshape((Npixel, Npixel))

plt.figure(figsize=(5,5))
plt.imshow(ui > 0.5, cmap="gray_r")
plt.axis("off")
plt.tight_layout()
date_time_tag = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
output_path = "output/"
os.makedirs(output_path, exist_ok=True)
plt.savefig(f'{output_path}gray_scott_result_{date_time_tag}_.pdf')
plt.show()

print("Result image saved as gray_scott_result.pdf.")
